# Sustainable Aviation Fuel Case Study

This case study examines five pathways for sustainable aviation fuel production from a hydroprocessed esters and fatty acids (HEFA) biorefinery. The five pathways use different feedstocks:

* HEFA Camelina
* HEFA Castor
* HEFA Jatropha
* HEFA Pennycress
* HEFA Yellow Grease

and produce different quantities of jet blendstock along with gasoline, diesel, and propane co-products.

We examine how these biorefinery technologies might respond to investment in co-product revenue (CPR), overall process efficiency (OPE), and fatty acid composition (FAC). All technologies are modeled under a default scenario that includes no additional investment, called *State of Technology*. Then there are three levels of investments possible in each research category, resulting in the *Incremental Progress*, *Moderate Progress*, and *Leap Forward* investment scenarios.

### Dimensions and indices.

1.  Types of capital costs. 
    * For this study all capital costs are treated as one lump sum.
2.  Types of fixed cost. 
    * For this study all fixed (non-input-related) costs are treated as one lump sum.
3.  Inputs to the process. 
    * Feedstock *(feedstock type varies by technology)*
    * Raw Material *(includes all chemical, material, and energy inputs other than feedstock)*
4.  Outputs from the process. Not all technologies produce all outputs.
    * Jet (SAF)
    * Gasoline
    * Diesel
    * Propane
5.  Metrics. 
    * Jet GHG (kg CO2eq/gal)
    * Minimum jet selling price (MJSP)
6.  Parameters. 
    * Annual GHG (kg CO2eq)
    * Jet LHV (MJ/gal)
    * Discount Rate (unitless)
    * Depreciation Period (years)
    * Income Tax Rate (unitless)
    * All Capital (USD)
    * All Fixed (USD/year)
    * Diesel (gal/year)
    * Gasoline (gal/year)
    * Propane (gal/year)
    * Jet (gal/year)
    * Diesel Price (USD/gal)
    * Gasoline Price (USD/gal)
    * Propane Price (USD/gal)
    * Feedstock Price (USD/dry metric ton biomass or USD/dry ton oil)
    * Raw Material Price (USD/kg)

## Implement the technology model.

The implementation of a technology model consists of a capital cost function, a fixed cost function, a production function, and a metrics function.

See the [model](https://tyche.live/doc-src/formulation.html) documentation for more details.

The [src/technology/](../src/technology/) folder has examples of several technology models.

### Capital cost function.

The capital cost function takes the scale of the operations and the array of technology parameters as arguments and it returns the capital costs for the technology.

Implement the capital cost function for your technology in the file [tutorial/my_technology.py](./).

In [ ]:
def capital_cost(scale, parameter):
    """
    Capital cost function.

    Parameters
    ----------
    scale : float
      The scale of operation.
    parameter : array
      The technological parameterization.
    """
    _cap = parameter[5]

    # Stack the costs for each category into a single array that we return.
    return np.stack([_cap
    ])

### Fixed cost function.

The fixed cost function takes the scale of the operations and the array of technology parameters as arguments and it returns the fixed costs for the technology.

Implement the fixed cost function for your technology in the file [tutorial/my_technology.py](./).

In [ ]:
def fixed_cost(scale, parameter):
  """
  Fixed cost function.

  Parameters
  ----------
  scale : float
    The scale of operation.
  parameter : array
    The technological parameterization.
  """
  _fix = parameter[6]

  # Stack the costs for each category into a single array that we return.
  return np.stack([_fix
  ])

### Production function.

The production function takes the scale of the operations, the capital costs, the lifetime, the fixed costs, and the array of technology parameters as arguments and it returns the production (outputs) for the technology.

Implement the production function for your technology in the file [tutorial/my_technology.py](./).

In [ ]:
def production(scale, capital, lifetime, fixed, input, parameter):
  """
  Production function.

  Parameters
  ----------
  scale : float
    The scale of operation.
  capital : array
    Capital costs.
  lifetime : float
    Technology lifetime.
  fixed : array
    Fixed costs.
  input : array
    Input quantities.
  parameter : array
    The technological parameterization.
  """

  _out = parameter[12]

  # Stack the output for each category into a single array that we return.
  return np.stack([_out
  ])

### Metric function.

The metric function takes information on costs, inputs, outputs, and parameters and it returns the metrics for the technology.

Implement the metrics function for your technology in the file [tutorial/my_technology.py](./).

In [ ]:
def metrics(scale, capital, lifetime, fixed, input_raw, input, output_raw, output, cost, parameter):
  """
  Metrics function.

  Parameters
  ----------
  scale : float
    The scale of operation.
  capital : array
    Capital costs.
  lifetime : float
    Technology lifetime.
  fixed : array
    Fixed costs.
  input_raw : array
    Raw input quantities (before losses).
  input : array
    Input quantities.
  output_raw : array
    Raw output quantities (before losses).
  output : array
    Output quantities.
  cost : array
    Costs.
  parameter : array
    The technological parameterization.
  """

  # annual fossil GHG emissions, Units: kg CO2-eq/year
  ghg_foss_ann = parameter[0]

  # jet fuel LHV (MJ/gal)
  lhv = parameter[1]

  # Discount rate, Unitless
  dr = parameter[2]
  # Depreciation period for all equipment except utilities, Units: years
  dp = parameter[3]
  # Income tax rate, Units: years
  tr = parameter[4]
  # total capital cost, Units: USD
  cap = parameter[5]
  # Annual fixed (non-input) costs, Units: USD/year
  fix = parameter[6]

  # co-product amounts
  ddgs = parameter[7]
  elec = parameter[8]
  dsl = parameter[9]
  gas = parameter[10]
  ppn = parameter[11]
  jet = parameter[12]

  # co-product prices
  ddgs_p = parameter[13]
  elec_p = parameter[14]
  dsl_p = parameter[15]
  gas_p = parameter[16]
  ppn_p = parameter[17]

  # input prices
  fd_p = parameter[18]
  mat_p = parameter[19]

  # input amounts
  fd = input_raw[0]
  mat = input_raw[1]

  # equipment lifetime
  els = lifetime[0]

  mjsp_bench = parameter[20]
  ghg_bench = parameter[21]

  # FOSSIL GHG: kg CO2-eq/gal SAF
  ghg_foss = ghg_foss_ann / (jet * lhv)

  # MINIMUM FUEL SELLING PRICE: USD/gal SAF

  # total project investment, Units: USD
  tpi = cap

  # depreciation costs, units: USD/year
  dc = cap / dp

  # operating costs, units: USD/year
  oc = fd * fd_p + mat * mat_p + fix

  # tpi discount factor, Units: unitless
  df_tpi = (dr * (1 + dr) ** els) / ((1 + dr) ** els - 1)

  # total revenue from SAF sales, Units: USD/year
  br = ((1 - tr) * oc - tr * dc + df_tpi * tpi) / (1 - tr)

  # revenue from co-product sales, Units: USD/year
  cr = ddgs * ddgs_p + elec * elec_p + dsl * dsl_p + gas * gas_p + ppn * ppn_p

  # MJSP, Units: USD/gal SAF
  mjsp = (br + cr) / output[0]

  # Package results.
  return np.stack([ghg_bench - ghg_foss,
                   mjsp_bench - mjsp
  ])

## Simulate the base case or reference scenario.

Load the data. Compile the production and metric functions for each technology in the dataset. Compute an ensemble of 1000 simulations for the State of Technology (no investment) case.

In [ ]:
import os
import sys

sys.path.insert(1, os.path.abspath("../../src"))

import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import seaborn           as sb
import tyche             as ty

saf_loc = "../../tutorial/data/saf"

designs = ty.Designs(saf_loc)
designs.compile()

In [ ]:
designs.evaluate_scenarios(sample_count=1)

In [ ]:
saf_sot = designs.evaluate_scenarios(
    sample_count=1
).xs(
    "State of Technology",
    level = "Scenario",
    drop_level = False
)
saf_sot

## Simulate effects of R&D.

### Simulating implications of expert elicitation.

In [ ]:
saf_experts = designs.evaluate_scenarios(sample_count=1000)
saf_experts

In [ ]:
saf_experts.xs(
    ("Metric", "Reduction in MJSP"),
    level=("Variable","Index")
).reset_index(
).replace(
    "State of Technology",
    "State of\nTechnology"
).replace(
    "Incremental Progress",
    "Incremental\nProgress"
).replace(
    "Moderate Progress",
    "Moderate\nProgress"
).replace(
    "Leap Forward",
    "Leap\nForward"
)

#.replace(
#    to_replace=["State of Technology","Incremental Progress","Moderate Progress","Leap Forward"],
#    value=["State of\nTechnology","Incremental\nProgress","Moderate\nProgress","Leap\nForward"],
#    inplace=True
#)



In [ ]:
sb.catplot(
    x = "Scenario",
    y = "Value",
    col ="Technology",
    order = ["Incremental\nProgress",
             "Moderate\nProgress",
             "Leap\nForward"],
    data = saf_experts.xs(
        ("Metric", "Reduction in MJSP"),
        level=("Variable","Index")
    ).reset_index(
    ).replace(
        "State of Technology",
        "State of\nTechnology"
    ).replace(
        "Incremental Progress",
        "Incremental\nProgress"
    ).replace(
        "Moderate Progress",
        "Moderate\nProgress"
    ).replace(
        "Leap Forward",
        "Leap\nForward"
    ),
    kind="violin",
    split=True
).set(
    xlabel = None,
    title = None
).set_xticklabels(
    rotation=20,
    size=20
).set_ylabels(
    "MJSP Reduction\n(USD/gal)",
    size=20
);
pl.tight_layout()
pl.savefig('mjsp-sot-violin-alltechs.png')

In [ ]:
sb.catplot(
    x = "Scenario",
    y = "Value",
    col ="Technology",
    order = ["Incremental\nProgress",
             "Moderate\nProgress",
             "Leap\nForward"],
    data = saf_experts.xs(
        ("Metric", "Reduction in Jet GHG"),
        level=("Variable","Index")
    ).reset_index(
    ).replace(
        "State of Technology",
        "State of\nTechnology"
    ).replace(
        "Incremental Progress",
        "Incremental\nProgress"
    ).replace(
        "Moderate Progress",
        "Moderate\nProgress"
    ).replace(
        "Leap Forward",
        "Leap\nForward"
    ),
    kind="violin",
    split=True
).set(
    xlabel = None,
    title = None
).set_xticklabels(
    rotation=20,
    size=20
).set_ylabels(
    "GHG Reduction\n(gCO2eq/gal)",
    size=20
);
pl.tight_layout()
pl.savefig('ghg-sot-violin-alltechs.png')

## Analyzing investments.

In [ ]:
saf_designs = ty.Designs(saf_loc)
saf_investments = ty.Investments(saf_loc)
saf_designs.compile()
tranche_results = saf_investments.evaluate_tranches(saf_designs,
                                                    sample_count=1000)

Filter down to one technology, since the dataset has seven and the functions aren't written to handle more than one at a time. Then step through and optimize each technology individually for MJSP.

In [ ]:
opt_results = []

techs = tranche_results.summary.index.get_level_values('Technology').unique()

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator,
        sense = 'min',
    )
    metric_max = optimizer.max_metrics()
    w = optimizer.opt_slsqp(
        "MJSP",
        total_amount = 5000000
    )
    print(f"{i} - {w[1]}")
    opt_dict = {
        'Technology': i,
        'Optimizer status': w[1],
        'Investment amounts': w[2],
        'Optimum metrics': w[3],
        'Optimization sense' : w[5]
    }
    opt_results.append(
        opt_dict
    )
    del tranche_tech_results, evaluator, optimizer, metric_max, w
opt_results

#### Find the largest reduction in MJSP if only 5M USD can be spent and only 2M USD can be spent in each category.

In [ ]:
opt_results = []

techs = tranche_results.summary.index.get_level_values('Technology').unique()

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator
    )
    metric_max = optimizer.max_metrics()
    w = optimizer.maximize_slsqp(
        "Reduction in MJSP",
        total_amount = 5000000,
        max_amount = pd.Series(
            [2000000, 2000000, 2000000],
            index=["OPE Investment Only", "CPR Investment Only", "FAC Investment Only"]
        )
    )
    print(f"{i} - {w[1]}")
    opt_dict = {
        'Technology': i,
        'Optimizer status': w[1],
        'Investment amounts': w[2],
        'Optimum metrics': w[3],
        'Maximum metrics': metric_max
    }
    opt_results.append(
        opt_dict
    )
opt_results

In [ ]:
opt_results = []

i = 'HEFA Camelina'

tranche_tech_results = tranche_results.summary[
    tranche_results.summary.index.get_level_values('Technology') == i
]
evaluator = ty.Evaluator(
    saf_investments.tranches, tranche_tech_results
)
optimizer = ty.EpsilonConstraintOptimizer(
    evaluator
)
metric_max = optimizer.max_metrics()

w = optimizer.maximize_slsqp(
    "Reduction in Jet GHG",
    total_amount = 5000000,
    max_amount = pd.Series(
        [2000000, 2000000, 2000000],
        index=["OPE Investment Only",
               "CPR Investment Only",
               "FAC Investment Only"]
    )
)
print(f"{i} - {w[1]}")
opt_dict = {
    'Technology': i,
    'Optimizer status': w[1],
    'Investment amounts': w[2],
    'Optimum metrics': w[3],
    'Maximum metrics': metric_max
}

opt_dict

Use the epsilon constraint method to explore the trade-off between reducing MJSP and reducing jet GHG for the HEFA castor technology. For this demonstration, the \\$2M cap on funding for each research area is removed, leaving only the \\$5M total funding cap.

In [ ]:
opt_results = []

i = 'HEFA Castor'

tranche_tech_results = tranche_results.summary[
    tranche_results.summary.index.get_level_values('Technology') == i
]
evaluator = ty.Evaluator(
    saf_investments.tranches, tranche_tech_results
)
optimizer = ty.EpsilonConstraintOptimizer(
    evaluator
)
metric_max = optimizer.max_metrics()

ghg = optimizer.maximize_slsqp(
    "Reduction in Jet GHG",
    total_amount = 5000000
)
print(f"{ghg[1]}")
opt_results = [
    {
        'Optimum metrics': ghg[3]
    }
]
mjsp = optimizer.maximize_slsqp(
    "Reduction in MJSP",
    total_amount = 5000000
)
print(f"{mjsp[1]}")
opt_results.append([
    {
        'Optimum metrics': mjsp[3]
    }
])

opt_results

In [ ]:
optimizer._max_metrics

In [ ]:
w_ghg = ty.Waterfall(
    amounts = ghg.amounts,
    evaluator = evaluator,
    data = saf_loc,
    metric = 'Reduction in Jet GHG',
)
w_ghg.cascade_permutations()

w_mjsp = ty.Waterfall(
    amounts = mjsp.amounts,
    evaluator = evaluator,
    data = saf_loc,
    metric = 'Reduction in MJSP',
)
w_mjsp.cascade_permutations()

In [ ]:
def pct_diff(bench, reduction):
    return 100.0 * (bench - reduction) / bench

def find_const(bench, pct_diff):
    return bench * (1 - pct_diff/100.0)

In [ ]:
mjsp_bench = 10.32857
ghg_bench = 0.053

maxmjsp_mjsp = pct_diff(mjsp_bench, mjsp[3]['Reduction in MJSP'])
maxmjsp_ghg = pct_diff(ghg_bench, mjsp[3]['Reduction in Jet GHG'])
maxghg_mjsp = pct_diff(mjsp_bench, ghg[3]['Reduction in MJSP'])
maxghg_ghg = pct_diff(ghg_bench, ghg[3]['Reduction in Jet GHG'])

In [ ]:
maxmjsp_ghg

In [ ]:
maxmjsp_mjsp

In [ ]:
maxghg_mjsp

In [ ]:
maxghg_ghg

In [ ]:
eps_results = []

i = 'HEFA Castor'

tranche_tech_results = tranche_results.summary[
    tranche_results.summary.index.get_level_values('Technology') == i
]
evaluator = ty.Evaluator(
    saf_investments.tranches, tranche_tech_results
)
optimizer = ty.EpsilonConstraintOptimizer(
    evaluator
)

for j in np.arange(100, 320, 20):

    w = optimizer.maximize_slsqp(
        "Reduction in MJSP",
        total_amount = 5000000,
        min_metric   = pd.Series(
            [find_const(ghg_bench, j)],
            name="Value",
            index = ["Reduction in Jet GHG"]
        ),
    )
    print(f"{j}, {find_const(ghg_bench, j)}: {w[1]}")
    
    if w[1] == 'Optimization terminated successfully':
        eps_results.append({
            'GHG Percentage of Bench': j,
            'MJSP Percentage of Bench': pct_diff(mjsp_bench, w[3][3]),
            'Jet GHG Reduction': find_const(ghg_bench, j),
            'Optimum MJSP Reduction': w[3][3]
        })

eps_results